### Build up features, join to clusters

In [1]:
import numpy as np
import os, sys, json
import PIL
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from ast import literal_eval
import warnings
warnings.filterwarnings('ignore')

sys.path.append('..')
%load_ext autoreload
%autoreload 2

from modules.overlaputils import foo

crop_dir = '../../../rf-chess-data/cropped_verify/'

In [2]:
!ls ../data/rf-annotate/

data_aug_v1.csv  data_v1.1.csv	data_v1.csv


In [3]:
!ls ../../../rf-chess-data/

cropped_v1	eda_1.ipynb  roboflow	 verify.files
cropped_verify	eda_2.ipynb  v1.1.files


### Setup Twin Pairs

In [4]:
df = pd.read_csv('../data/rf-annotate/data_v1.csv')  # '../data/rf-annotate/data_v1.1.csv'
df['bbox'] = df['bbox'].map(lambda x: literal_eval(x))

In [5]:
bq = foo(df, 'black-queen')

bq.shape

# bq.head(3)

(87, 17)

In [6]:
amts = bq.groupby('twin_set_cluster_obj').agg('count').iloc[:,1]
# amts

In [7]:
max_show = 99 
matched = amts[amts > 1]
unmatched = amts[amts == 1]

iters = matched.sort_values(ascending=False).index[:max_show]

In [8]:
iters

Index(['[7, 11, 13, 14, 21, 23, 28, 40, 50, 51, 52, 54, 56, 62, 63, 70, 79, 81, 83]',
       '[2, 18, 29, 37, 38, 46, 48, 57, 59, 66, 67, 72, 73, 78]',
       '[5, 6, 16, 20, 30, 32, 33, 39, 47, 60, 69, 86]',
       '[1, 4, 8, 10, 22, 43, 44, 53, 65, 68, 75, 84]',
       '[9, 17, 19, 35, 55, 71, 85]', '[3, 24, 26]', '[45, 58]', '[42, 49]',
       '[27, 74]'],
      dtype='object', name='twin_set_cluster_obj')

In [9]:
literal_eval(iters[0])

[7, 11, 13, 14, 21, 23, 28, 40, 50, 51, 52, 54, 56, 62, 63, 70, 79, 81, 83]

In [10]:
bq.iloc[:4, 13:]

,twin_sets,twin_sets_obj,twin_set_cluster,twin_set_cluster_obj
1,[0],[0],[0],[0]
19,"[1, 4, 8, 10, 22, 43, 44, 53, 65, 68, 75, 84]","[1, 4, 8, 10, 22, 43, 44, 53, 65, 68, 75, 84]","[1, 4, 8, 10, 22, 43, 44, 53, 65, 68, 75, 84]","[1, 4, 8, 10, 22, 43, 44, 53, 65, 68, 75, 84]"
83,"[2, 67]","[2, 67]","[2, 18, 29, 37, 38, 46, 48, 57, 59, 66, 67, 72...","[2, 18, 29, 37, 38, 46, 48, 57, 59, 66, 67, 72..."
188,"[3, 24, 26]","[3, 24, 26]","[3, 24, 26]","[3, 24, 26]"


### Setup Data / Learner

In [11]:
import torch
from fastai.vision.all import *

In [12]:
def piece_class_parse(fn): 
    fn = fn.split('_')[1]
    fn = fn.split('.')[0]
    return fn

train_path = Path('../../../rf-chess-data/cropped_v1/')

Crop = Resize(128, ResizeMethod.Pad, pad_mode='reflection')

train_dl = ImageDataLoaders.from_name_func(
                    
                    train_path, 
                    get_image_files(train_path),                 
                    label_func=piece_class_parse, 
                    item_tfms=Crop,
                    
                    # these are for an evaluation run
                    shuffle_train=False,
                    valid_pct=.0,
                    drop_last=False,
                    num_workers=0,
                    bs=16,
                    )

In [13]:
learn = cnn_learner(train_dl, resnet50, metrics=[], normalize=False)

PATH = '../models/base-learner-6.pth'

model = learn.model
model.load_state_dict(torch.load(PATH))
model.eval();
learn.model = model

In [14]:
new_model = torch.nn.modules.container.Sequential(model[0], model[1][:2])
learn.model = new_model

### Build Dataset from Loader

In [15]:
train = train_dl[0]

In [16]:
items = train.items.copy()
idxs  = train.get_idxs().copy()

In [17]:
num_batches = len(train.items) / train.bs
num_batches

179.375

In [42]:
# counter = 0
# for x,y in train:
#     counter += 1
#     if counter > 50: break
# counter

In [43]:
counter = 0
y_all = torch.tensor([])
feats_all = torch.tensor([])

In [ ]:
for x,y in train:
    
    _feats = new_model(x)
    
    y_all =     torch.cat((y_all, y))
    feats_all = torch.cat((feats_all, _feats))
    
    counter += 1
    #if counter > 10: break
    if counter > int(num_batches) + 1: break
counter

In [ ]:
feats_all.shape

In [ ]:
y_all.shape